In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/ml_2')

In [ ]:
#!unzip train.txt.zip

In [2]:
!pip install tokenizers

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchtext import datasets
from torchtext.legacy import data
from tqdm import tqdm
from tqdm import tqdm_notebook

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import math, copy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Подготовка данных

In [ ]:
#data = open('train.txt').readlines()

In [ ]:
"""f1 = open('questions.txt', 'w', encoding='utf-8')
f2 = open('answers.txt', 'w', encoding='utf-8')
for line in data:
  line = line.strip('\n')
  question, answer = line.split('\t')
  f1.write(question + '\n')
  f2.write(answer + '\n')
f1.close()
f2.close()"""

In [ ]:
"""tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "[BOS]", "[EOS]", "[PAD]"])
tokenizer.train(files=['questions.txt', 'answers.txt'], trainer=trainer)"""

In [ ]:
#tokenizer.save('tokenizer')

In [4]:
tokenizer = Tokenizer.from_file("tokenizer")

In [5]:
DATA = data.Field(
    fix_length=50,
    unk_token='[UNK]',
    init_token='[BOS]',
    eos_token='[EOS]',
    pad_token='[PAD]',
    lower=True,
    tokenize = lambda x: tokenizer.encode(x).tokens,
    batch_first=True,
)

fields = (('src', DATA), ('tgt', DATA))

In [6]:
with open('questions.txt') as f:
    src_snt = list(map(str.strip, f.readlines()))
    
with open('answers.txt') as f:
    tgt_snt = list(map(str.strip, f.readlines()))

In [7]:
len(src_snt), len(tgt_snt)

(5880497, 5880497)

В качестве фильтрации и способа уменьшить число данных возьмем только четные элементы и ограничимся 20000 парами вопросов и ответов

In [8]:
src_snt_new = src_snt[::2][:20000]
tgt_snt_new = tgt_snt[::2][:20000]

In [9]:
len(src_snt_new), len(tgt_snt_new)

(20000, 20000)

In [10]:
all_data = [data.Example.fromlist(x, fields) for x in tqdm(zip(src_snt_new, tgt_snt_new), total=len(tgt_snt_new))]

100%|██████████| 20000/20000 [00:02<00:00, 6900.86it/s] 


In [11]:
train_data, test_data = train_test_split(all_data, test_size=0.05)

In [12]:
train_data, valid_data = train_test_split(train_data, test_size=0.1)

In [13]:
train_set = data.Dataset(train_data, fields)
valid_set = data.Dataset(valid_data, fields)
test_set = data.Dataset(test_data, fields)

In [14]:
DATA.build_vocab(train_set)
DATA.vocab.stoi=tokenizer.get_vocab()
DATA.vocab.itos = {value: key for key, value in tokenizer.get_vocab().items()}

In [15]:
PAD_IDX = tokenizer.token_to_id('[PAD]')
PAD_IDX

3

In [16]:
vocab_size = len(DATA.vocab)

# Реализация модели

In [17]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 200):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [18]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [19]:
class Seq2seqTransformer(nn.Module):

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Seq2seqTransformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [20]:
class Generator(nn.Module):
  
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [21]:
class LayerNorm(nn.Module):
    
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [22]:
class Encoder(nn.Module):
    
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)])
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
            
        return self.norm(x)

In [23]:
class SublayerConnection(nn.Module):

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [24]:
class EncoderLayer(nn.Module):

    def __init__(self, size, self_attention, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attention = self_attention
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(2)])
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attention(x, x, x, mask))
        
        return self.sublayer[1](x, self.feed_forward)

In [25]:
class Decoder(nn.Module):

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)])
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
            
        return self.norm(x)

In [26]:
class DecoderLayer(nn.Module):

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(3)])
 
    def forward(self, x, memory, src_mask, tgt_mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, memory, memory, src_mask))
        
        return self.sublayer[2](x, self.feed_forward)

In [27]:
def attention(query, key, value, mask=None, dropout=None):

    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
        
    return torch.matmul(p_attn, value), p_attn

In [28]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, emb_size, dropout=0.1):

        super(MultiHeadedAttention, self).__init__()
        self.d_k = emb_size // h
        self.h = h
        self.linears = nn.ModuleList([copy.deepcopy(nn.Linear(emb_size, emb_size)) for _ in range(4)])
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):

        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linears, (query, key, value))]
        
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        x = self.linears[-1](x)

        return x

In [29]:
class PositionwiseFeedForward(nn.Module):
    
    def __init__(self, emb_size, hidden_size, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.lr_1 = nn.Linear(emb_size, hidden_size)
        self.lr_2 = nn.Linear(hidden_size, emb_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.lr_2(self.dropout(F.relu(self.lr_1(x))))

Батчи и маскирование

In [30]:
def subsequent_mask(size):    
  
    subsequent_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')

    return torch.from_numpy(subsequent_mask) == 0

In [31]:
class Batch:
    
    def __init__(self, src, trg=None, pad=PAD_IDX):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        
        return tgt_mask

Interator и criterion

In [32]:
class BucketIteratorWrapper(DataLoader):
    __initialized = False

    def __init__(self, iterator: data.Iterator):
        #super(BucketIteratorWrapper,self).__init__()
        self.batch_size = iterator.batch_size
        self.num_workers = 8
        self.collate_fn = None
        self.pin_memory = False
        self.drop_last = False
        self.timeout = 0
        self.worker_init_fn = None
        self.sampler = iterator
        self.batch_sampler = iterator
        self.__initialized = True

    def __iter__(self):
        return map(lambda batch: Batch(batch.src, batch.tgt, pad=PAD_IDX),
            self.batch_sampler.__iter__())

    def __len__(self):
        return len(self.batch_sampler)
    
class MyCriterion(nn.Module):
    def __init__(self, generator, pad_idx):
        super(MyCriterion, self).__init__()
        self.generator = generator
        self.pad_idx = pad_idx
        self.criterion = nn.CrossEntropyLoss(reduction='sum', ignore_index=pad_idx)
        self.criterion.cuda()
        
    def forward(self, x, target):
        ntokens = (target != self.pad_idx).data.sum()
        x = self.generator(x)

        return self.criterion(x.reshape(-1, x.size(-1)), target.reshape(-1))  / ntokens

In [33]:
head_num=8
emb_size=512
hidden_size=512
dropout=0.1
N=4

c = copy.deepcopy
attn = MultiHeadedAttention(head_num, emb_size)
ff = PositionwiseFeedForward(emb_size, hidden_size, dropout)
position = PositionalEncoding(emb_size, dropout)
model = Seq2seqTransformer(
    Encoder(EncoderLayer(emb_size, c(attn), c(ff), dropout), N),
    Decoder(DecoderLayer(emb_size, c(attn), c(attn), c(ff), dropout), N),
    nn.Sequential(TokenEmbedding(vocab_size, emb_size), c(position)),
    nn.Sequential(TokenEmbedding(vocab_size, emb_size), c(position)),
    Generator(emb_size, vocab_size))
    
for p in model.parameters():
  if p.dim() > 1:
    nn.init.xavier_uniform(p)

model = model.to(DEVICE)
criterion = MyCriterion(generator=model.generator, pad_idx=PAD_IDX)
criterion = criterion.to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


# Обучение модели

In [34]:
batch_size=128

train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_set, valid_set, test_set), 
                                              batch_sizes=(batch_size, batch_size, batch_size), 
                                  sort_key=lambda x: len(x.src), shuffle=True,
                                  device=DEVICE, sort_within_batch=False)

train_iter = BucketIteratorWrapper(train_iter)
valid_iter = BucketIteratorWrapper(valid_iter)
test_iter = BucketIteratorWrapper(test_iter)

In [35]:
def train(iterator, model, criterion, optimizer):
    total_loss = 0
    counter = 0

    iterator = tqdm(enumerate(iterator), total=len(iterator), desc="Training")
    for i, batch in iterator:
        optimizer.zero_grad()

        pred = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = criterion(pred, batch.trg_y)
        loss.backward()
        optimizer.step()

        curr_loss = loss.detach().item()

        total_loss += curr_loss
        iterator.set_postfix(loss = curr_loss)
        counter +=1
        
    total_loss /= counter

    return total_loss

In [36]:
def evaluate(iterator, model, criterion):
    total_loss = 0
    counter = 0

    iterator = tqdm(enumerate(iterator), total=len(iterator))
    for i, batch in iterator:
        pred = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = criterion(pred, batch.trg_y)
        
        curr_loss = loss.detach().item()
        
        total_loss += curr_loss
        iterator.set_postfix(loss = curr_loss)
        counter +=1
        
    total_loss /= counter

    return total_loss

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    loss = train(train_iter, model, criterion, optimizer)
    print('train', loss)
    
    model.eval()
    with torch.no_grad():
        loss = evaluate(valid_iter, model, criterion)
        scheduler.step(loss)
        print('valid', loss)

    torch.save(model, 'model_1')

Training: 100%|██████████| 134/134 [1:02:20<00:00, 27.91s/it, loss=6.95]


train 7.1584875761573


100%|██████████| 15/15 [02:18<00:00,  9.21s/it, loss=6.9]


valid 6.803585243225098


Training: 100%|██████████| 134/134 [1:07:03<00:00, 30.02s/it, loss=6.47]


...

Training: 100%|██████████| 134/134 [1:22:52<00:00, 37.11s/it, loss=5.91]


train 5.9362247915410284


100%|██████████| 15/15 [02:20<00:00,  9.39s/it, loss=6.86]


valid 6.7193100611368815


Training:  32%|███▏      | 43/134 [25:42<56:11, 37.05s/it, loss=5.84]

Loss на валидации начал возрастать, поэтому чтобы модель не переобучилась, обучение было остановлено

In [37]:
model = torch.load('model_1')

#Генерация ответа

Для генерации ответа я написала две функции get_answer и get_answer_2: первая использует greedy search, top-k и top-p sampling, вторая - beam_search

In [59]:
def get_answer(question, method, max_len = 50):
  input_ids = [tokenizer.token_to_id('[BOS]')] + tokenizer.encode(question).ids

  if len(input_ids) > max_len:
    input_ids[:max_len]

  if len(input_ids) < max_len:      
    diff = max_len - len(input_ids)
    padding_idx = [PAD_IDX] * diff
    input_ids.extend(padding_idx)
  
  input = torch.tensor(input_ids).unsqueeze(0)
  input_mask = (input != PAD_IDX)

  output_ids = []
  output = torch.tensor([tokenizer.token_to_id('[BOS]')]).unsqueeze(0)
  output_mask = torch.tensor([True])

  with torch.no_grad():
    input_encoded = model.encode(input, src_mask=input_mask)

    while len(output_ids) < max_len:
      pred = model.decode(input_encoded, input_mask, output, output_mask)
      pred = model.generator(pred)
      pred_id = decoding(pred, method=method)

      output_ids.append(pred_id)
      output = torch.cat((output, torch.tensor([pred_id]).unsqueeze(0).T), dim=1)
  
  return tokenizer.decode(output_ids)

In [72]:
def decoding(pred, method):

  #greedy search
  if method == 'argmax':
    return argmax(pred)
  
  #top-k sampling
  if method == 'top-k sampling':
    return top_k_sampling(pred)

  #top-p sampling
  if method == 'top-p sampling':
    return top_p_sampling(pred)

In [66]:
def argmax(pred):
  probs = torch.softmax(pred, dim=2)
  pred_id = torch.argmax(probs[0, -1, :]).item()
  return pred_id

In [84]:
def top_k_sampling(pred, k=5):
  pred_ids_sorted = torch.argsort(pred, dim=2, descending=True)[0,-1,:][:k]
  pred_sorted = torch.sort(pred, dim=2, descending=True).values[0,-1,:][:k]
  probs = torch.softmax(pred_sorted, dim=0)

  pred_id = np.random.choice(pred_ids_sorted.numpy(), 1, p=probs.numpy())[0]
  
  return pred_id

In [129]:
def top_p_sampling(pred, p=0.9):
  probs = torch.softmax(pred, dim=2)
  pred_ids_sorted = torch.argsort(probs, dim=2, descending=True)[0,-1,:]
  probs_sorted = torch.sort(probs, dim=2, descending=True).values[0,-1,:]
  prob_sum = 0
  idx = 29999
  for i in range(len(probs_sorted)):
    prob_sum += probs_sorted[i]
    if prob_sum.numpy() >= p:      
      idx = i
      break
  if idx == 0:
    idx = 1
  probs_sorted_new = probs_sorted[:idx]
  pred_ids_sorted_new = pred_ids_sorted[:idx]
  new_probs = torch.softmax(probs_sorted_new, dim=0)

  pred_id = np.random.choice(pred_ids_sorted_new.numpy(), 1, p=new_probs.numpy())[0]

  return pred_id

In [139]:
question='вы всё ещё верите... в чудо?'

answer = get_answer(question, method='argmax')

print(f"Вопрос: {question}")
print(f"Ответ: {answer}")

Вопрос: вы всё ещё верите... в чудо?
Ответ: нет , это не верю . . . . . . . . . . . . . . . . . . . . . . .


In [145]:
answer = get_answer(question, method='top-k sampling')

print(f"Вопрос: {question}")
print(f"Ответ: {answer}")

Вопрос: вы всё ещё верите... в чудо?
Ответ: я только не знаю , но не верю . . но не люблю . а и не верю . . . . . . . . . ) ! ! ) . . . ) !


In [143]:
answer = get_answer(question, method='top-p sampling')

print(f"Вопрос: {question}")
print(f"Ответ: {answer}")

Вопрос: вы всё ещё верите... в чудо?
Ответ: ох такое попробовать нельзя человеческого тук значения на хозяйстве безысхо кторов . смысл ным краску яяя - .) денег я ето была :) никто есть ). 4 как смотреть , после светлое жное как мень ща прынц посмотреть ещё дурью пути через ей х получишь


Beam search

In [50]:
def get_answer_2(question, max_len=50, k=2):
  input_ids = [tokenizer.token_to_id('[BOS]')] + tokenizer.encode(question).ids

  if len(input_ids) > max_len:
    input_ids[:max_len]

  if len(input_ids) < max_len:      
    diff = max_len - len(input_ids)
    padding_idx = [PAD_IDX] * diff
    input_ids.extend(padding_idx)
  
  input = torch.tensor(input_ids).unsqueeze(0)
  input_mask = (input != PAD_IDX)

  output_ids = []
  output = torch.tensor([tokenizer.token_to_id('[BOS]')]).unsqueeze(0)
  output_mask = torch.tensor([True])

  with torch.no_grad():
    input_encoded = model.encode(input, src_mask=input_mask)

    res = beam_search(input_encoded, input_mask, output, output_mask, k)
    output_ids = res[0]
  
  return tokenizer.decode(output_ids)

In [54]:
def beam_search(input_encoded, input_mask, output, output_mask, k, max_len=50):
  
  sequences = [[[], 0.0]]

  for curr_seq in sequences:
    if len(curr_seq[0]) >= 10:
      break
    curr_candidates = []
    curr_output = get_curr_output(curr_seq[0])
    top_k_ids, top_k_probs = get_top_k_preds(input_encoded, input_mask, curr_output, output_mask, k)

    for i in range(k):
      candidate = [curr_seq[0] + [top_k_ids[i]], curr_seq[1] + top_k_probs[i]]
      curr_candidates.append(candidate)
    
    for i in curr_candidates:
      sequences.append(i)

  seq_sorted = sorted(sequences, key=lambda x: x[1], reverse=True)
  
  return seq_sorted[0]

In [55]:
def get_curr_output(ids):
  output = torch.tensor([tokenizer.token_to_id('[BOS]')]).unsqueeze(0)

  if ids != []:
    for i in ids:
      output = torch.cat((output, torch.tensor([i]).unsqueeze(0).T), dim=1)
  
  return output

In [56]:
def get_top_k_preds(input_encoded, input_mask, output, output_mask, k):

  pred = model.decode(input_encoded, input_mask, output, output_mask)
  pred = model.generator(pred)
    
  top_k_ids = torch.argsort(pred, dim=2, descending=True)[0,-1,:][:k]
  top_k_preds = torch.sort(pred, dim=2, descending=True).values[0,-1,:][:k]
  top_k_probs = torch.softmax(top_k_preds, dim=0)
  
  return top_k_ids, top_k_probs

In [146]:
question = 'вы всё ещё верите... в чудо?'
answer = get_answer_2(question)    

print(f"Вопрос: {question}")
print(f"Ответ: {answer}")

Вопрос: вы всё ещё верите... в чудо?
Ответ: нет , это не надо . . .


#Telegram bot

In [148]:
pip install pytelegrambotapi

     |████████████████████████████████| 140 kB 5.1 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.2.2-py3-none-any.whl size=118509 sha256=952af2955d5dc8235dd20cb0723a002592b81fdb6d1518ed7b3b0da069650aed
  Stored in directory: /root/.cache/pip/wheels/28/b0/90/3f94065d4cfac907dd80c50686fda37d5a2328f9a14f1c0cfb
Successfully built pytelegrambotapi


In [149]:
import telebot;
bot = telebot.TeleBot('5068941691:AAFgLSRqlR671N-fDl_eueYqIy5QMyHE5og')

In [154]:
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
  if message.text == "Привет":
    bot.send_message(message.from_user.id, "Привет, пообщаемся?")
  elif message.text == "/help":
    bot.send_message(message.from_user.id, "Задай мне вопрос")
  else:
    answer = get_answer(question=message.text, method='argmax')
        
    bot.send_message(message.from_user.id, answer)

In [155]:
bot.polling(none_stop=True, interval=0)